Una empresa de paquetería quiere repartir en una ciudad un conjunto de paquetes en distintas 
casas. Para ello quiere saber cual es la ruta más corta para repartir todos los paquetes. Un 
paquete tiene un peso y una prioridad de entrega (generados aleatoriamente) y un repartidor 
tiene una capacidad máxima de cuantos paquetes puede llevar al mismo tiempo, basado en 
el peso (generado aleatoriamente). <br>

Dado un grafo grande de casas (definido por el mismo equipo), donde los nodos representan 
una casa a donde se puede entregar un paquete, y existe una distancia entre cada nodo. 
Cuando el repartidor se quede sin paquetes que entregar debe regresar a la matriz de la 
empresa para verificar si existen paquetes por entregar y así hasta quedarse sin paquetes. El 
objetivo es empezar en el nodo de la empresa y minimizar el costo total de las rutas para 
entregar eficientemente los paquetes en el menor tiempo y con la menor distancia recorrida. 

Posibles datos de entrada: <br>
• Conjunto de paquetes con peso, prioridad y nodo de entrega.<br>
• Vehículo de entregas con peso máximo de paquetes que puede llevar.<br>
• Mapa de rutas que contiene el grafo de información.<br>

Objetivos:<br>
• Minimizar la distancia total recorrida por los vehículos.<br>
• Minimizar los tiempos de entrega de los paquetes.<br>
• Entregar todos los paquetes.<br>

Restricciones:<br>
• Los vehículos no deben superar su capacidad máxima<br>
• Todos los paquetes deben ser entregados<br>
• Los nodos del grafo no están todos interconectados, puedes necesitar ir a otra casa 
antes de llegar al objetivo.<br>

Sugerencias adicionales:<br>
• Empezar con un algoritmo genético aplicado a encontrar una ruta eficiente en un grafo
de ida y vuelta a varios puntos, y luego agregar las restricciones de paquetes y peso.<br>
• Probar diferentes configuraciones de parámetros para la configuración del genotipo, la 
evaluación de la solución, la generación de una nueva población o el factor de 
mutación.

In [1]:
# Lista de paquetes con prioridad y peso.
# El índice de la lista representa el número de paquete.
# Y cada paquete está conformado por [prioridad, peso]
listaPaquetes = [[8, 31], #1
                 [10, 40], #2
                 [2, 21], #3
                 [5, 39], #4
                 [7, 42], #5
                 [1, 6], #6
                 [3, 28], #7
                 [9, 18], #8
                 [4, 13], #9
                 [6, 38]] #10

# NOTA: Acorde la definición del problema, cada peso y prioridad de los paquetes tiene que ser
#       generado aleatoriamente. La lista anterior solo es un ejemplo.

# El problema no dice que el número de paquetes debe ser aleatorio,
# entonces podemos decir que siempre serán 10 paquetes

In [38]:
# Grafo completo de las ciudades, las mismas que se usaron en el problema de viajeros.
# En este caso está almacenado como diccionario.
# Se considerará Bucharest como el lugar de la empresa, ya que tenemos las distancias euclidianas de Bucharest
# a las demás ciudades.

# El grafo es un diccionario que contiene un nodo representando una ciudad, y cada nodo es un diccionario que almacena
# las ciudades a la que está conectada, que contiene la distancia entre ciudades.

# Grafo de distancias entre ciudades
grafo = {
    "Arad": {
        "Zerind": 75, # Distancia entre Arad a Zerind.
        "Sibiu": 140, # Distancia entre Arad a Sibiu.
        "Timisoara": 118 # Distancia entre Arad a Timisoara.
    },
    "Zerind": {
        "Oradea": 71,
        "Arad": 75
    },
    "Oradea": {
        "Sibiu": 151,
        "Zerind": 71
    },
    "Timisoara": {
        "Lugoj": 111,
        "Arad": 118
    },
    "Lugoj": {
        "Mehadia": 70,
        "Timisoara": 111
    },
    "Mehadia": {
        "Drobeta": 75,
        "Lugoj": 70
    },
    "Drobeta": {
        "Craiova": 120,
        "Mehadia": 75
    },
    "Sibiu": {
        "Fagaras": 99,
        "Rimnicu Vilcea": 80,
        "Oradea": 151,
        "Arad": 140
    },
    "Fagaras": {
        "Bucharest": 211,
        "Sibiu": 99
    },
    "Rimnicu Vilcea": {
        "Pitesti": 97,
        "Craiova": 146,
        "Sibiu": 80
    },
    "Craiova": {
        "Pitesti": 138,
        "Drobeta": 120
    },
    "Pitesti": {
        "Bucharest": 101,
        "Rimnicu Vilcea": 97
    },
    "Bucharest": {  # <----- Considerar Bucharest como la EMPRESA DE PAQUETERÍA
        "Giurgiu": 90,
        "Urziceni": 85,
        "Fagaras": 211
    },
    "Giurgiu": {
        "Bucharest": 90
    },
    "Urziceni": {
        "Hirsova": 98,
        "Vaslui": 142,
        "Bucharest": 85
    },
    "Hirsova": {
        "Eforie": 86,
        "Urziceni": 98
    },
    "Eforie": {
        "Hirsova": 86
    },
    "Vaslui": {
        "Iasi": 92,
        "Urziceni": 142
    },
    "Iasi": {
        "Neamt": 87,
        "Vaslui": 92
    },
    "Neamt": {
        "Iasi": 87
    }
}

# Diccionario de distancias euclidianas
distancias_euclidianas = {
    "Arad": 366,    # Distancia euclidiana (linea recta) entre Arad y Bucharest.
    "Zerind": 374,
    "Oradea": 380,
    "Timisoara": 329,
    "Lugoj": 244,
    "Mehadia": 241,
    "Drobeta": 242,
    "Sibiu": 253,
    "Rimnicu Vilcea": 193,
    "Craiova": 160,
    "Fagaras": 176,
    "Pitesti": 100,
    "Bucharest": 0, # <--------- EMPRESA DE PAQUETERIA
    "Giurgiu": 77,
    "Urziceni": 80,
    "Hirsova": 151,
    "Eforie": 161,
    "Neamt": 234,
    "Iasi": 226,
    "Vaslui": 199
}


In [3]:
ciudades = list(grafo.keys())
print(ciudades)

['Arad', 'Zerind', 'Oradea', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta', 'Sibiu', 'Rimnicu Vilcea', 'Craiova', 'Pitesti', 'Bucharest', 'Giurgiu', 'Urziceni', 'Hirsova', 'Eforie', 'Vaslui', 'Iasi', 'Neamt']


In [14]:
import random

# Genera un paquete con peso, prioridad y destino aleatorios.
def generarPaquete():
    # Ejemplo de ciudades
    ciudades = list(grafo.keys())

    # Generar un paquete
    peso = random.randint(1, 50)  # Peso aleatorio del paquete
    prioridad = random.randint(1, 10)  # Prioridad aleatoria del 1 al 10, entre más mayor más prioridad
    nodo_entrega = random.choice(ciudades)  # Nodo de entrega aleatorio

    paquete = {
        "peso": peso,
        "prioridad": prioridad,
        "nodo_entrega": nodo_entrega
    }
    
    return paquete


In [16]:
generarPaquete()

{'peso': 40, 'prioridad': 9, 'nodo_entrega': 'Craiova'}

In [17]:
# Genera una lista de 10 paquetes aleatorios.
# En este caso 10 paquetes, ya que el problema no especifica
# que el número de paquetes existentes deba ser aleatorio.
def generarListaPaquetes():
    lista_paquetes = []
    cantidad = 10
    for _ in range(cantidad):
        paquete = generarPaquete()
        lista_paquetes.append(paquete)
    return lista_paquetes

In [19]:
generarListaPaquetes()

[{'peso': 21, 'prioridad': 7, 'nodo_entrega': 'Pitesti'},
 {'peso': 40, 'prioridad': 10, 'nodo_entrega': 'Mehadia'},
 {'peso': 5, 'prioridad': 10, 'nodo_entrega': 'Lugoj'},
 {'peso': 19, 'prioridad': 10, 'nodo_entrega': 'Rimnicu Vilcea'},
 {'peso': 7, 'prioridad': 3, 'nodo_entrega': 'Bucharest'},
 {'peso': 36, 'prioridad': 6, 'nodo_entrega': 'Iasi'},
 {'peso': 30, 'prioridad': 9, 'nodo_entrega': 'Arad'},
 {'peso': 37, 'prioridad': 9, 'nodo_entrega': 'Bucharest'},
 {'peso': 19, 'prioridad': 6, 'nodo_entrega': 'Vaslui'},
 {'peso': 5, 'prioridad': 5, 'nodo_entrega': 'Craiova'}]

In [ ]:
def generarRepartidor(lista_paquetes):
    # Definir la capacidad de carga aleatoria del repartidor
    capacidad_carga = random.randint(50, 150)  # Peso máximo que puede llevar

    # Ubicación inicial del repartidor, que es la ciudad de la empresa
    ubicacion_inicial = "Bucharest"

    # Inicializar la lista de paquetes que puede llevar
    paquetes = []
    peso_total = 0

    # Seleccionar paquetes que el repartidor puede llevar sin exceder su capacidad
    for paquete in lista_paquetes[:]:  # Hacemos una copia de la lista para evitar problemas al modificarla
        if peso_total + paquete['peso'] <= capacidad_carga:
            paquetes.append(paquete)
            peso_total += paquete['peso']
            lista_paquetes.remove(paquete)  # Remover el paquete de la lista original

    # Obtener la ruta a partir de los nodos de entrega de los paquetes
    ruta = [paquete['nodo_entrega'] for paquete in paquetes]

    # Crear el repartidor como un diccionario
    repartidor = {
        "capacidad": capacidad_carga,
        "ubicacion": ubicacion_inicial,
        "carga": paquetes,  # Campo para almacenar los paquetes
        "peso_total": peso_total,
        "ruta": ruta    # Lista de ciudades de entrega
    }

    return repartidor

In [31]:
# Prueba para comparar que el repartidor tome correctamente los paquetes
# en base a la capacidad y que éstos se salgan de la lista de paquetes.
test = generarListaPaquetes()
print(test)
test2 = generarRepartidor(test)
print(test2)
print(test)

[{'peso': 7, 'prioridad': 9, 'nodo_entrega': 'Drobeta'}, {'peso': 14, 'prioridad': 5, 'nodo_entrega': 'Arad'}, {'peso': 4, 'prioridad': 5, 'nodo_entrega': 'Mehadia'}, {'peso': 39, 'prioridad': 9, 'nodo_entrega': 'Pitesti'}, {'peso': 20, 'prioridad': 8, 'nodo_entrega': 'Neamt'}, {'peso': 11, 'prioridad': 6, 'nodo_entrega': 'Eforie'}, {'peso': 41, 'prioridad': 3, 'nodo_entrega': 'Rimnicu Vilcea'}, {'peso': 22, 'prioridad': 7, 'nodo_entrega': 'Zerind'}, {'peso': 33, 'prioridad': 9, 'nodo_entrega': 'Eforie'}, {'peso': 18, 'prioridad': 5, 'nodo_entrega': 'Lugoj'}]
{'capacidad': 128, 'ubicacion': 'Bucharest', 'carga': [{'peso': 7, 'prioridad': 9, 'nodo_entrega': 'Drobeta'}, {'peso': 14, 'prioridad': 5, 'nodo_entrega': 'Arad'}, {'peso': 4, 'prioridad': 5, 'nodo_entrega': 'Mehadia'}, {'peso': 39, 'prioridad': 9, 'nodo_entrega': 'Pitesti'}, {'peso': 20, 'prioridad': 8, 'nodo_entrega': 'Neamt'}, {'peso': 11, 'prioridad': 6, 'nodo_entrega': 'Eforie'}, {'peso': 22, 'prioridad': 7, 'nodo_entrega': 